In [0]:
import pandas as pd
from matplotlib import pyplot
from sklearn.externals import joblib
import numpy as np
import datetime
import pickle

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path= "./drive/My Drive/uk dale dataset/channel_2.dat"

In [0]:
def string_to_timestamp(string):
    date_time_obj = datetime.datetime.strptime(string, '%Y-%m-%d %H:%M:%S')
    timestamp = date_time_obj.timestamp()
    return timestamp

In [0]:
class data_prep():
    def __init__(self, path):
      self.df = pd.read_csv(path, sep='\s+', header=None)

    def resampler(self,time = "5min"):
      dataframe = self.df.set_index([0])
      dataframe.index = pd.to_datetime(dataframe.index,unit = "s")
      resample = dataframe.resample(time)
      self.resampled_data = resample.sum()

    def get_resampled_with_timestamp(self):
      self.final_data = self.resampled_data.reset_index()

    def get_final_data(self):
      self.resampler()
      self.get_resampled_with_timestamp()
      return self.final_data

In [0]:
prep = data_prep(path)
data = prep.get_final_data()

In [99]:
data.head(5)

,0,1
0,2012-11-09 22:25:00,409
1,2012-11-09 22:30:00,965
2,2012-11-09 22:35:00,967
3,2012-11-09 22:40:00,917
4,2012-11-09 22:45:00,380


In [0]:
class to_serialised_on_off():
    def __init__(self, data):
      self.data = data
      self.min_ = 0
      self.avg_ = 0
      self.max_ = 0
      self.find_min_max(flag = 1)

    def append_to_df(self, val):
      self.data.append(val)
      self.find_min_max()

    def find_min_max(self, flag = 0):
      if(flag == 0):
        val = self.data.loc[-1][-1]
        if( val > self.max_ ):
          self.max_ = val
        elif( val < self.min_ ):
          self.min_ = val
        self.avg_ += val/self.data.shape[0]
      elif(flag):
        self.max_ = max(self.data.loc[:][-1])
        self.min_ = min(self.data.loc[:][-1])
        self.avg_ = np.mean(self.data.loc[:][-1].values(), axis = 1)

      calc_thresh()

    def calc_thresh(self):
      self.thresh = (self.max_ - self.min_)/self.avg_

    def on_off(self, preds):
      of = []
      for val in preds:
        if(val < thresh):
          of.append(0)
        else:
          of.append(1)

      return of